In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.utils.data as data

import torch.optim as optim

import torchvision
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import visdom
import numpy as np

In [ ]:
vis = visdom.Visdom()
vis.close(env="main")

In [ ]:
def plt_tracker(loss_plot, loss_value, num):
    vis.line(X=np.stack(np.arange(num, num+1)),
             Y=np.stack([loss_value]),
             win=loss_plot,
             update='append'
             )

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [ ]:
transform = transforms.Compose([transforms.ToTensor()])
train = dsets.CIFAR10(root='./data',train=True, transform=transform,download=True)
print(train.train_data.shape)
mean = train.train_data.mean(axis=(0,1,2))/255
std = train.train_data.std(axis=(0,1,2))/255


transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean,std)])

train = dsets.CIFAR10(root='./data',train=True, transform=transform,download=True)
test = dsets.CIFAR10(root='./data', train=False, transform=transform, download=True)

In [ ]:
train_loader = data.DataLoader(dataset=train, batch_size=64, num_workers=4)
test_loader = data.DataLoader(dataset=test, batch_size=64, num_workers=4)

In [ ]:
class small_VGG(nn.Module):
    def __init__(self):
        super(small_VGG,self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(3,64,3,padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.Conv2d(64,64,3,padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.Conv2d(64,64,3,padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            )
        self.pool = nn.MaxPool2d(2)
        self.layer2 = nn.Sequential(
            nn.Conv2d(64,128,3,padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.Conv2d(128,128,3,padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.Conv2d(128,128,3,padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            )
        self.layer3 = nn.Sequential(
            nn.Conv2d(128,256,3,padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.Conv2d(256,256,3,padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.Conv2d(256,256,3,padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            )
        self.fc1 = nn.Linear(256*8*8,256)
        self.fc2 = nn.Linear(256,10)
        
        
    def forward(self,x):
        x = self.layer(x)
        x = self.pool(x)
        x = self.layer2(x)
        x = self.pool(x)
        x = self.layer3(x)
        x = x.view(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
net = small_VGG().to(device)

In [ ]:
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
criterion = nn.CrossEntropyLoss().to(device)

lr_sche = optim.lr_scheduler.StepLR(optimizer,step_size=5,gamma=0.5)

In [ ]:
loss_plot = vis.line(Y=torch.Tensor(1).zero_())

In [ ]:
epoch = 15
data_len = len(train_loader)

print(data_len)

for e in range(epoch):
    print('epoch : {}'.format(e))
    running_loss = 0.0
    check_loss = 0.0
    
    lr_sche.step()
    for num, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.cuda().to(device)
        
        optimizer.zero_grad()       
        
        out = net(inputs)
        loss = criterion(out,labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        check_loss += loss.item()
        
        if (num % 100 == 1) and (num != 1):
            check_loss = check_loss / 100
            print("loss : {}".format(check_loss))
            plt_tracker(loss_plot,(check_loss), e *len(train_loader) + num)
            check_loss = 0.0 
            
        if num % len(train_loader) == (len(train_loader) - 1):
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, num + 1, running_loss / len(train_loader)))
            running_loss = 0.0
        
    

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
print('Accuracy of the network on the 10000 thest images: {} '.format(100 * correct / total))